In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import os
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib
import time
import datetime
from IPython import display
import random



data_directory = 'D:\\Deletar\\pictures'
test_directory = 'D:\\Deletar\\pictures_conv'
tf.random.set_seed(42) 
SIZE = [1024,1024]

In [2]:
def resize_image(data):
    resized_img = tf.image.resize(
    images=data,
    size=SIZE,
    method=tf.image.ResizeMethod.BILINEAR,
    preserve_aspect_ratio=False,
    antialias=False,
    name=None
    )    
    return resized_img




def save_f(img,filename,contI):
    dir_save = ""
    if (contI % 5 != 0):
        dir_save = f"{test_directory}\\train\\{filename}.jpg"
    else:
        dir_save = f"{test_directory}\\test\\{filename}.jpg"
    tf.keras.preprocessing.image.save_img(dir_save, img)
    
#if (os.path.exists(os.path.join(f"{data_directory}\\{folder}",filename+".jpg"))):    

def load_images_from_folder(folder):
    images = []
    cont = 0
    cont2 = 0    
    for folder in os.listdir(folder):
        for filename in os.listdir(os.path.join(f"{data_directory}\\{folder}")):        
            cont2 = cont2 + 1
            if (not os.path.exists(os.path.join(f"{data_directory}\\{folder}\\",filename+".jpg"))):                        
                try:
                    img1 = resize_image(plt.imread(os.path.join(f"{data_directory}\\{folder}\\",filename)))
                    print(img1)
                    img2 =  tf.image.rgb_to_grayscale(img1)
                    img2 = np.concatenate((img2,img2,img2),axis=-1)
                    img_merged = tf.concat([img1, img2], 1)
                    if img_merged is not None:
                        save_f(img_merged,filename,cont2)
                        """ images.append(img_merged) """
                except:
                        cont = cont + 1
                        print(f"Error in {cont}")
            print(cont2)
    return images

X_test = load_images_from_folder(data_directory)

plt.imshow(np.array(X_test[0], dtype=int))


tf.Tensor(
[[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 ...

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]], shape=(1024, 1024, 3), dtype=float32)
1
tf.Tensor(
[[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 2